In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
seed = 11667
n_collect = 200
cvs = np.concatenate([np.linspace(0, 0.2, 11), np.linspace(0.2, 0.5, 13)[1:]])
coverage0 = 0.2

folders = [
#     'dy_hi', 
#     'dy_lo',
    'dy_ladd'
]

In [3]:
def collect_fux(pars_post, coverage=coverage0, alg="VSC"):    
    m = ModelIntv()
    
    mss = list()
    mss_stats = list()

    for i, pars in enumerate(pars_post):
        y1, pars = m.find_baseline(pars, 2022)

        _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv={}, polished=True)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        
        ms_stats.update({'Key': i, 'Scenario': 'Baseline', 'FollowUp': 0, 'Duration': 0})
        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            for dur in np.linspace(0, 4, 9):
                intv = {'FullACF': {'Coverage': coverage, 'FollowUp': n_fu, 'Duration': dur, 'ScreenAlg': alg}}
                
                _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv=intv, polished=True)
                
                ms = ms.reset_index().assign(Key=i, FollowUp=n_fu, Duration=dur)
                mss.append(ms)

                ms_stats.update({'Key': i, 'Scenario': 'Intervention', 'FollowUp': n_fu, 'Duration': dur})
                mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [4]:
for folder in folders:
    print(folder)
    for cov in [0.1, 0.2, 0.5]:
        print(folder, cov)
        
        rd.seed(seed)
        
        out_path = f'out/{folder}'
        pars_post = json.load(open(f'{out_path}/Post.json', 'r'))

        sel = rd.choice(list(range(len(pars_post))), n_collect)
        sel.sort()
        pars_post = [pars_post[i] for i in sel]

        mss, mss_stats = collect_fux(pars_post, coverage=cov)
        mss_stats = mss_stats.fillna(0)
        mss.to_csv(f'{out_path}/Sim_VulFu_fudur_{cov:.2f}.csv')
        mss_stats.to_csv(f'{out_path}/Sim_VulFu_fudur_{cov:.2f}_stats.csv')

dy_ladd
dy_ladd 0.1
dy_ladd 0.2
dy_ladd 0.5


In [5]:
mss_stats

,Inc,Mor,ACF_Footfall,ACF_Screened,ACF_Yield,ACF_TP,ACF_Fl,ACF_Sl,ACF_TPT,ACF_fu_Footfall,...,ACF_Alt_Yield,ACF_Alt_TP,ACF_Alt_Fl,ACF_Alt_Sl,ACF_Alt_TPT,Key,Scenario,FollowUp,Duration,Coverage
0,216340.982594,21842.043876,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Baseline,0,0.0,0.5
1,191868.433652,18275.593592,1.267349e+07,1.264178e+07,36574.806221,20778.542652,20778.542652,0.000000,15796.263569,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,0.0,0.5
2,192251.951775,18331.322733,1.267349e+07,1.264171e+07,36637.549180,20842.600358,20842.600358,0.000000,15794.948822,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,0.5,0.5
3,192252.498694,18331.385194,1.267349e+07,1.264171e+07,36637.594921,20842.648328,20842.648328,0.000000,15794.946593,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,1.0,0.5
4,192252.342957,18331.383453,1.267349e+07,1.264171e+07,36637.617348,20842.670680,20842.670680,0.000000,15794.946668,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,1.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,161793.866205,15412.947662,1.267349e+07,1.264109e+07,59948.877612,17079.588605,16860.023876,219.564729,42869.289007,1.007674e+07,...,0.0,0.0,0.0,0.0,0.0,199,Intervention,6,2.0,0.5
12796,161347.067954,15355.369674,1.267349e+07,1.264093e+07,59953.044037,16990.892721,16745.112808,245.779913,42962.151316,1.099910e+07,...,0.0,0.0,0.0,0.0,0.0,199,Intervention,6,2.5,0.5
12797,161002.164939,15310.754416,1.267349e+07,1.264081e+07,59955.641171,16922.173108,16656.031282,266.141826,43033.468063,1.171060e+07,...,0.0,0.0,0.0,0.0,0.0,199,Intervention,6,3.0,0.5
12798,160729.907794,15275.498820,1.267349e+07,1.264072e+07,59957.763134,16867.993129,16585.516031,282.477098,43089.770005,1.227542e+07,...,0.0,0.0,0.0,0.0,0.0,199,Intervention,6,3.5,0.5
